<h1>LeakyWaveML</h1>
<h3>Grey Slot Schema</h3>
<br>
This code contains the functions and methods used to generate the training data for the grey scheme and to re-run the numerical simulations for the slots generated by the model. Note that COMSOL Multiphysics is required.

In [76]:
import random
import mph
import numpy as np
import matplotlib.pyplot as plt
import os    
import time
from tqdm.notebook import tqdm
import pickle
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import tensorflow as tf

path = 'D:/Josh/Fall 2021/ML-LWA/partialslotsmodel_transuse_50only.mph'

scattering = [1,2,5,6,7,51,53,56,57]

<h3>Helper Functions</h3>

In [77]:
def start_client(path):
    '''
    Starts COMSOL client and initializes API access
    '''
    
    global model
    global pymodel
    global client
    client = mph.start(version = '5.3a')
    pymodel = client.load(path)
    model = pymodel.java
    
def save_model(path):
    '''
    Saves model
    '''
    
    model.save(path)
    
def generate_image(random_pattern):
    '''
    Arranges slot design into form appropriate for export (flip due to how element numbering works in COMSOL)
    '''
    
    return np.flip(np.array(random_pattern) / 2)
    
def simulate_random():
    '''
    Generates a random slot design so that the number of slots is roughly even and runs its numerical simulation
    '''
    
    global start_count
    
    vals = np.random.randint(3, size = 36)

    while (sum(vals) / len(vals) < 0.9) or (sum(vals) / len(vals) > 1.1):
        vals = np.random.randint(3, size = 36)
        
    vals = list(vals)

    bucket_list = [0, 1, 2]
    
    buckets = []
    
    for out_index,bucket in enumerate(bucket_list):
        out_list = [index + 18 for index,slot in enumerate(vals) if slot == bucket]
        buckets.append([x-1 if x == 18 else x for x in out_list])

    boundlist = ['1', '3', '5']

    for i in np.arange(0,3):
        model.physics("emw").feature("trans" + boundlist[i]).selection().set(buckets[i]);

    model.geom("geom1").runAll();
    model.mesh().run();
    model.sol("sol1").runAll();
    
    model.result("pg4").set("data", 'dset1')
    model.result("pg4").feature("ff1").run()
    model.result().export("plot1").set("filename", save_str + '.csv')
    model.result().export("plot1").set("plotgroup", "pg4")
    model.result().export("plot1").set("plot", "ff1")
    
    model.result().export("plot1").run()
    
    return vals

def simulate(vals):
    '''
    Similar to simulate_random but accepts a slot design (vals) rather than generating a new one
    '''
    
    bucket_list = [0, 1, 2]
    
    buckets = []
    
    for out_index,bucket in enumerate(bucket_list):
        out_list = [index + 18 for index,slot in enumerate(vals) if slot == bucket]
        buckets.append([x-1 if x == 18 else x for x in out_list])
        
    boundlist = ['1', '3', '5']

    for i in np.arange(0,3):
        model.physics("emw").feature("trans" + boundlist[i]).selection().set(buckets[i]);

    model.geom("geom1").runAll();
    model.mesh().run();
    model.sol("sol1").runAll();
    
    model.result("pg4").set("data", 'dset1')
    model.result("pg4").feature("ff1").run()
    model.result().export("plot1").set("filename", save_str + '.csv')
    model.result().export("plot1").set("plotgroup", "pg4")
    model.result().export("plot1").set("plot", "ff1")
    
    model.result().export("plot1").run()
    
    return vals

<h3>Training Data Generation</h3>

In [ ]:
for instance in tqdm(np.arange(40)):
    img_list = []

    save_str = 'C:/Users/THzAbacus/Documents/Josh/LeakyWaveML/comsol_results/3grey/' + str(int(time.time()))

    start_client(path)
    start_time = time.time()
    for i in tqdm(np.arange(500)):
        goi = simulate_random()
        img = generate_image(goi)
        img_list.append(img)

    print(time.time() - start_time, 'seconds')

    with open(save_str + '.pkl', 'wb') as file:
        pickle.dump(img_list, file)

In [51]:
save_model('D:/Josh/Fall 2021/ML-LWA/partialslotsmodel_transuse_50onlyTEST2.mph')

<h3>Validation -- re-running simulations for the slots the model generates</h3>

In [78]:
with open('1d/peaks_to_slot/3grey/results/test_data627.pkl', 'rb') as pkl_file:
    slots, true, preds, _ = pickle.load(pkl_file)
    
preds = np.where((preds > 0) & (preds <= 0.333), 0, preds)
preds = np.where((preds > 0.333) & (preds <= 0.666), 0.50, preds)
preds = np.where((preds > 0.666), 1.00, preds)

In [79]:
save_str = 'C:/Users/THzAbacus/Documents/Josh/LeakyWaveML/1d/peaks_to_slot/3grey/results/test627'

start_client(path)
start_time = time.time()
for i in tqdm(np.arange(500)):
      
    goi = simulate([int(x) for x in np.flip(preds[i]) * 2])

print(time.time() - start_time, 'seconds')

  0%|          | 0/500 [00:00<?, ?it/s]

32565.178386211395 seconds
